In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
TSUN = 4.9169e-6
gbfisher_columns = ['f','colatitude','longitude','amplitude','inclination','polarization','phase','fdot','fddot','sigma_df/f','sigma_colatitude','sigma_longitude','sigma_dA/A','sigma_inclination','sigma_polarization','sigma_phase','sigma_dfdot/fdot','sigma_dfddot/fddot','sigma_Omega','SNR','b','l','mchirp','distance'] 
gbfisher = pd.read_table('../../ldasoft/gbfisher_example/gbfisher_parameters.dat',delimiter=' ', header=None,index_col=False,names=gbfisher_columns)
gbfisher['inclination'] = 90 - abs(np.degrees(gbfisher['inclination']) - 90)
gbfisher['sigma_mchirp'] = (1/(5*TSUN))*((5/3)**(3/5))*(np.pi**(-8/5))*(gbfisher['f']**(-11/5))*(gbfisher['fdot']**(3/5))*((121*(gbfisher['sigma_df/f']**2) + 9*(gbfisher['sigma_dfdot/fdot']**2))**(1/2))
gbfisher.head()

In [ ]:
def readData(raw, index, inFunc=None):
    x = []
    for pos, line in enumerate(raw):
        values = [float(x) for x in line.split()]
        if inFunc==None:
            x.append(values[index])
        else:
            x.append(inFunc(values[index]))
    return x

In [ ]:
prior_types = ['em','gw']
prefix = 'out'

for c,p in enumerate(prior_types):
    binDir = os.path.join(os.getcwd(),f'../{prefix}-{p}prior')
    for binary in os.listdir(binDir):
        rawData = []
        B_n = int(binary[-3:])
        with open(os.path.join(binDir,os.path.join(binary,os.path.join('inc','2-post_equal_weights.dat'))),'r') as raw:
            rawData = np.concatenate([rawData,readData(raw,0,inFunc=lambda x: None)])
        if (B_n == 1 and c==0):
            vdata = pd.DataFrame(np.sort(rawData))
            vdata.insert(0, 'B_n', B_n)
            vdata.insert(0, 'type', f'{p}')
        else:
            raw_vdata = pd.DataFrame(np.sort(rawData))
            raw_vdata.insert(0, 'B_n', B_n)
            raw_vdata.insert(0, 'type', f'{p}')
            vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
# prior_types = ['em','gw']
# exclude = ['fdot','inc']

# for c,p in enumerate(prior_types):
#     binDir = os.path.join(os.getcwd(),f'../out-{p}prior')
#     for binary in os.listdir(binDir):
#         rawData = []
#         for folder in os.listdir(os.path.join(binDir,binary)):
#             if os.path.isdir(os.path.join(binDir,os.path.join(binary,folder))) & (not folder in exclude):
#                 B_n = int(re.search('binary(.+?)row',folder).group(1))
#                 with open(os.path.join(binDir,os.path.join(binary,os.path.join(folder,'post_equal_weights.dat'))),'r') as raw:
#                     rawData = np.concatenate([rawData,readData(raw,4,inFunc=lambda x: 90-abs(x-90))])
#         if (B_n == 1 and c==0):
#             vdata = pd.DataFrame(np.sort(rawData))
#             vdata.insert(0, 'B_n', B_n)
#             vdata.insert(0, 'type', f'{p}')
#         else:
#             raw_vdata = pd.DataFrame(np.sort(rawData))
#             raw_vdata.insert(0, 'B_n', B_n)
#             raw_vdata.insert(0, 'type', f'{p}')
#             vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
chainDir = '../../data/results/'
trvs = []

for binary in os.listdir(chainDir):
    rawData = []
    B_n = int(re.search('binary(.*)',binary).group(1))
    with open(os.path.join(chainDir,os.path.join(binary,'chains/dimension_chain.dat.1')),'r') as raw:
        rawData = np.concatenate([rawData,readData(raw,5,inFunc=lambda x: 90-abs(np.degrees(np.arccos(x))-90))])
    raw_vdata = pd.DataFrame(np.sort(rawData))
    raw_vdata.insert(0, 'B_n', B_n)
    raw_vdata.insert(0, 'type', 'gbf')
    vdata = pd.concat([vdata,raw_vdata], axis=0)

In [ ]:
trvs = list(gbfisher['inclination'])
vdata = vdata.rename(columns={0:'value'})
vdata.insert(3, 'true_value', 0)
vdata['true_value'] = np.asarray(trvs)[vdata['B_n']-1]
vdata = vdata.sort_values(['true_value','type','value'])
vdata['value'] = vdata['value'] - vdata['true_value']
vdata['B_n'] = vdata['B_n'].map(dict(zip(vdata['B_n'].unique(),np.arange(1,len(vdata['B_n'].unique())+1))))

In [ ]:
b_raw = np.array([np.unique(c).tolist() for c in [vdata.loc[(vdata['B_n'] == b), 'B_n'] for b in vdata['B_n'].unique()]])
b_n = np.unique(b_raw[b_raw.astype(bool)].tolist()).tolist()
gbf = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gbf'),'value'] for b in b_n]
em = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'em'), 'value'] for b in b_n]
gw = [vdata.loc[(vdata['B_n'] == b) & (vdata['type'] == 'gw'), 'value'] for b in b_n]
tv = vdata['true_value'].unique()

In [ ]:
cut = [0,1,2,3,4]
qgbf = [[0.0001,0.9999] for i in range(len(cut))]
qem = [[0.0001,0.9999] for i in range(len(cut))]
qgw = [[0.0001,0.9999] for i in range(len(cut))]
wt = 1.75
#######################################################
fig = plt.figure(figsize=(36,18))
ax = fig.add_subplot(111)
ctv = tv[cut]

sgbf = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gbf])[cut])]
cgbf = [list(l) for l in [s[s.between(s.quantile(qgbf[i][0]),s.quantile(qgbf[i][1]))] for i,s in enumerate(sgbf)]]
sem = [pd.Series(l) for l in (np.asarray([s.tolist() for s in em])[cut])]
cem = [list(l) for l in [s[s.between(s.quantile(qem[i][0]),s.quantile(qem[i][1]))] for i,s in enumerate(sem)]]
sgw = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gw])[cut])]
cgw = [list(l) for l in [s[s.between(s.quantile(qgw[i][0]),s.quantile(qgw[i][1]))] for i,s in enumerate(sgw)]]

mgbf = [np.percentile(list(m),50) for m in sgbf]
hgbf = [np.percentile(list(m),90) for m in sgbf]
lgbf = [np.percentile(list(m),10) for m in sgbf]
mem = [np.percentile(list(m),50) for m in sem]
hem = [np.percentile(list(m),90) for m in sem]
lem = [np.percentile(list(m),10) for m in sem]
mgw = [np.percentile(list(m),50) for m in sgw]
hgw = [np.percentile(list(m),90) for m in sgw]
lgw = [np.percentile(list(m),10) for m in sgw]

vpgbf = plt.violinplot(cgbf, positions=ctv, widths=wt, showextrema=False)
for pc in vpgbf['bodies']:
    pc.set_facecolor('#20BB00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))
plt.hlines(mgbf,ctv-wt/2,ctv,linewidth=2.0)
# plt.hlines(lgbf,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
# plt.hlines(hgbf,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

vpem = plt.violinplot(cem, positions=ctv, widths=wt, showextrema=False)
for pc in vpem['bodies']:
    pc.set_facecolor('#E67E00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))
plt.hlines(mem,ctv-wt/2,ctv,linewidth=2.0) 
# plt.hlines(lem,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
# plt.hlines(hem,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

vpgw = plt.violinplot(cgw, positions=ctv, widths=wt, showextrema=False)
for pc in vpgw['bodies']:
    pc.set_facecolor('#226EF1')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],np.mean(pc.get_paths()[0].vertices[:, 0]),np.inf)
plt.hlines(mgw,ctv+wt/2,ctv,linewidth=2.0)
# plt.hlines(lgw,ctv+0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
# plt.hlines(hgw,ctv+0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

plt.legend([vpgbf['bodies'][0],vpem['bodies'][0],vpgw['bodies'][0]],['GW Prior','EM Posterior','GW+EM Posterior'],loc='upper right',prop={'size': 24})
plt.title('Residual Inclination vs True Inclination',fontsize=40,pad=10)
plt.xlabel('True Inclination (degrees)',fontsize=30)
plt.ylabel('Residual Inclination (degrees)',fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()

In [ ]:
cut = [0,1,2,3,4]
qgbf = [[0.0001,0.9999] for i in range(len(cut))]
qem = [[0.0001,0.9999] for i in range(len(cut))]
wt = 2
#######################################################
fig = plt.figure(figsize=(36,18))
ax = fig.add_subplot(111)
ctv = tv[cut]

sgbf = [pd.Series(l) for l in (np.asarray([s.tolist() for s in gbf])[cut])]
cgbf = [list(l) for l in [s[s.between(s.quantile(qgbf[i][0]),s.quantile(qgbf[i][1]))] for i,s in enumerate(sgbf)]]
sem = [pd.Series(l) for l in (np.asarray([s.tolist() for s in em])[cut])]
cem = [list(l) for l in [s[s.between(s.quantile(qem[i][0]),s.quantile(qem[i][1]))] for i,s in enumerate(sem)]]

mgbf = [np.percentile(list(m),50) for m in sgbf]
hgbf = [np.percentile(list(m),90) for m in sgbf]
lgbf = [np.percentile(list(m),10) for m in sgbf]
mem = [np.percentile(list(m),50) for m in sem]
hem = [np.percentile(list(m),90) for m in sem]
lem = [np.percentile(list(m),10) for m in sem]

vpgbf = plt.violinplot(cgbf, positions=ctv, widths=wt, showextrema=False)
for pc in vpgbf['bodies']:
    pc.set_facecolor('#20BB00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],-np.inf,np.mean(pc.get_paths()[0].vertices[:, 0]))
plt.hlines(mgbf,ctv-wt/2,ctv,linewidth=2.0)
# plt.hlines(lgbf,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
# plt.hlines(hgbf,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

vpem = plt.violinplot(cem, positions=ctv, widths=wt, showextrema=False)
for pc in vpem['bodies']:
    pc.set_facecolor('#E67E00')
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)
    pc.get_paths()[0].vertices[:, 0] = np.clip(pc.get_paths()[0].vertices[:, 0],np.mean(pc.get_paths()[0].vertices[:, 0]),np.inf)
plt.hlines(mem,ctv+wt/2,ctv,linewidth=2.0) 
# plt.hlines(lem,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')
# plt.hlines(hem,ctv-0.89*wt/4,ctv,linewidth=1.0,linestyle='--')

plt.legend([vpgbf['bodies'][0],vpem['bodies'][0]],['GW Prior','EM Posterior',''],loc='upper right',prop={'size': 24})
plt.title('Residual Inclination vs True Inclination',fontsize=40,pad=10)
plt.xlabel('True Inclination (degrees)',fontsize=30)
plt.ylabel('Residual Inclination (degrees)',fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()